In [1]:
import time 

import numpy as np
import pinocchio as pin
from tqdm import tqdm

from projectyl.dynamics.armmodel import ArmRobot
from projectyl.dynamics.meshcat_viewer_wrapper import MeshcatVisualizer

Let's create a robot arm

In [2]:
arm_robot = ArmRobot(upper_arm_length=0.3, forearm_length=0.25)

In [3]:
viz = MeshcatVisualizer(arm_robot)
viz.display(arm_robot.q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
# Iterate through all the joints in the robot's model
for i in range(arm_robot.model.njoints):
    joint_name = arm_robot.model.names[i]
    joint_model = arm_robot.model.joints[i]

    joint_type = joint_model.shortname()

    parent_joint_id = arm_robot.model.parents[i]
    parent_joint_name = arm_robot.model.names[parent_joint_id]

    # Print joint information
    print("Joint ID:", i)
    print("Joint Name:", joint_name)
    print("Joint Type:", joint_type)
    print("Parent Joint ID:", parent_joint_id)
    print("Parent Joint Name:", parent_joint_name)
    print()

Joint ID: 0
Joint Name: universe
Joint Type: JointModelRX
Parent Joint ID: 0
Parent Joint Name: universe

Joint ID: 1
Joint Name: shoulder
Joint Type: JointModelSpherical
Parent Joint ID: 0
Parent Joint Name: universe

Joint ID: 2
Joint Name: elbow
Joint Type: JointModelRUBY
Parent Joint ID: 1
Parent Joint Name: shoulder



In [5]:
# Iterate through all the frames in the robot's model
for i in range(len(arm_robot.model.frames)):
    frame = arm_robot.model.frames[i]
    parent_joint_id = frame.parent
    parent_joint_name = arm_robot.model.names[parent_joint_id]
    print("Frame ID:", i)
    print("Frame Name:", frame.name)
    print("Frame Type:", frame.type)
    print("Parent Joint ID:", parent_joint_id)
    print("Parent Joint name:", parent_joint_name)
    print()

Frame ID: 0
Frame Name: universe
Frame Type: FIXED_JOINT
Parent Joint ID: 0
Parent Joint name: universe

Frame ID: 1
Frame Name: end_effector
Frame Type: OP_FRAME
Parent Joint ID: 2
Parent Joint name: elbow



In [6]:
print("Initial configuration:", arm_robot.q0)
print("Config dim:", arm_robot.nq)
print("DoF:", arm_robot.nv)

Initial configuration: [0. 0. 0. 1. 1. 0.]
Config dim: 6
DoF: 4


Test : Random configuration

In [7]:
for _ in range(100):
    q = pin.randomConfiguration(arm_robot.model)
    viz.display(q)
    time.sleep(1e-2)

Test: Forward Kinematics

In [8]:
q = pin.randomConfiguration(arm_robot.model)
pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)

# pin.framesForwardKinematics is equivalent to first call ForwardKinematics, then 
# updateFramePlacements. Here not an issue since we have only one frame, but if 
# we add more, and we dont need FramePlacements a lot, may be usefull.

viz.display(q)

In [9]:
for i in range(arm_robot.model.njoints):
    joint_name = arm_robot.model.names[i]
    joint_position = arm_robot.data.oMi[i]
    print("Joint Name:", joint_name)
    print("Joint Position in World Frame:", joint_position)
    print()


Joint Name: universe
Joint Position in World Frame:   R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0


Joint Name: shoulder
Joint Position in World Frame:   R =
-0.237851 -0.738425 -0.630996
-0.467139 -0.482607  0.740859
-0.851592  0.470976 -0.230158
  p = 0 0 1


Joint Name: elbow
Joint Position in World Frame:   R =
 0.667646 -0.738425 0.0947502
-0.425096 -0.482607 -0.765757
 0.611181  0.470976 -0.636112
  p = -0.189299  0.222258  0.930952




In [10]:
for i in range(len(arm_robot.model.frames)):
    frame = arm_robot.model.frames[i]
    frame_placement = arm_robot.data.oMf[i]
    print("Frame Name:", frame.name)
    print("Frame Position in World Frame:", frame_placement)
    print()

Frame Name: universe
Frame Position in World Frame:   R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0


Frame Name: end_effector
Frame Position in World Frame:   R =
 0.667646 -0.738425 0.0947502
-0.425096 -0.482607 -0.765757
 0.611181  0.470976 -0.636112
  p = -0.165611 0.0308184  0.771924




In [11]:
# Test: confirm the position of the end effector relative to the elbow
joint_id = arm_robot.model.getJointId("elbow")
frame_id = arm_robot.model.getFrameId("end_effector")

joint_transform = arm_robot.data.oMi[joint_id]
frame_transform = arm_robot.data.oMf[frame_id]

relative_transform = joint_transform.inverse() * frame_transform
print("Relative position of the end effector with respect to the elbow, computed:", relative_transform)
print("Real value:", arm_robot.model.frames[frame_id].placement)

Relative position of the end effector with respect to the elbow, computed:   R =
           1  5.55112e-17 -2.77556e-16
 5.55112e-17            1 -1.66533e-16
-2.63678e-16 -1.66533e-16            1
  p =    0    0 0.25

Real value:   R =
1 0 0
0 1 0
0 0 1
  p =    0    0 0.25



Test: Inverse Kinematics

In [35]:
# Create first a target
q = pin.randomConfiguration(arm_robot.model)
pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)

target_position_end_effector = arm_robot.data.oMf[frame_id].copy()

In [36]:
# Print the target
viz.addBox("goal", [.1, .1, .1], [.1, .1, .5, .6])
viz.applyConfiguration("goal", target_position_end_effector)

In [37]:
# Robot initial position
q0 = arm_robot.q0
DT = 1e-2
viz.display(q0)

In [38]:
q = q0.copy()

for i in tqdm(range(500)):

    # end effector frame
    pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)
    oMeffector = arm_robot.data.oMf[frame_id]

    # Displacement between F_effector and F_goal
    effectorMgoal = oMeffector.inverse() * target_position_end_effector

    # error between the two
    effector_nu = pin.log(effectorMgoal).vector

    # Jacobian in the effector frame
    effector_Jeffector = pin.computeFrameJacobian(arm_robot.model, arm_robot.data, q, frame_id, pin.LOCAL)

    # Control law
    vq = np.linalg.pinv(effector_Jeffector) @ effector_nu

    # Integrate
    q = pin.integrate(arm_robot.model, q, vq*DT)

    viz.display(q)
    time.sleep(1e-3)

100%|██████████| 500/500 [00:05<00:00, 98.51it/s] 


In [39]:
# remove target
viz.delete("goal")

Test: Forward dynamics

In [77]:
# Random position, speed and torques of the robot
q = pin.randomConfiguration(arm_robot.model)
vq = np.random.rand(arm_robot.model.nv)

print("Initial joint position:", q)
print("Initial joint velocity:", vq)

M = pin.crba(arm_robot.model, arm_robot.data, q)

print("Mass matrix of the robot:\n", M)

Initial joint position: [-0.40730669  0.59324927  0.69329965 -0.03862855  0.99795106 -0.06398191]
Initial joint velocity: [0.29669337 0.28247928 0.41417722 0.94228816]
Mass matrix of the robot:
 [[0.21662102 0.         0.00370401 0.        ]
 [0.         0.21670466 0.         0.0586565 ]
 [0.00370401 0.         0.00213365 0.        ]
 [0.         0.0586565  0.         0.02123333]]


In [71]:
g = arm_robot.gravity(q)

In [72]:
print(q)

[ 0.13195617  0.87774071 -0.07065907 -0.45515503  0.79254134  0.60981819]
